<a href="https://colab.research.google.com/github/swaleha/Machine-Learning/blob/master/Dealing_with_Data_Linear_Separatrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.datasets import load_iris
import numpy as np
import keras 
np.random.seed(10)

Using TensorFlow backend.


In [2]:
iris = load_iris()
print(iris.keys())

X = iris['data'] # array([[5.1, 3.5, 1.4, 0.2],
                 #          [4.9, 3., 1.4, 0.2],
Y = iris['target'] # array ([0, 1, 2, 0, 1, .....])
names = iris['target_names']    # setosa, versicolor and virginica
feature_names = iris['feature_names'] # sepal length (cm)
                                      # sepal width (cm)
                                      # petal length (cm)
                                      # petal width (cm)

# Track a few example points
isamples = np.random.randint(len(Y), size=(5))  # array ([9, 125, 15, 64, 113]) <- random samples

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])


In [3]:
print(X.shape, Y.shape)
print(X[isamples])
print(Y[isamples])

(150, 4) (150,)
[[4.9 3.1 1.5 0.1]
 [7.2 3.2 6.  1.8]
 [5.7 4.4 1.5 0.4]
 [5.6 2.9 3.6 1.3]
 [5.7 2.5 5.  2. ]]
[0 2 0 1 2]


In [4]:
# Convert labels to categorical one-hot encoding
Ny = len(np.unique(Y))
Y = keras.utils.to_categorical(Y[:], num_classes = Ny)

# Y is np.ndarray now

print("X:", X[isamples,:])
print("Y:", Y[isamples])

X: [[4.9 3.1 1.5 0.1]
 [7.2 3.2 6.  1.8]
 [5.7 4.4 1.5 0.4]
 [5.6 2.9 3.6 1.3]
 [5.7 2.5 5.  2. ]]
Y: [[1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [5]:
# Train-test split (randomly into 80% - 20%)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 1)

print(X_train.shape)
print(X_test.shape)

(120, 4)
(30, 4)


In [7]:
# Data Scaling: scale X to zero-main and unit-variance

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)                              # computes mean and std
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
print(X_train)

[[ 0.31553662 -0.04578885  0.44767531  0.23380268]
 [ 2.2449325  -0.04578885  1.29769171  1.39742892]
 [-0.2873996  -1.24028061  0.05100098 -0.15407273]
 [ 0.67729835 -0.52358555  1.01435291  1.13884531]
 [-0.04622511 -0.52358555  0.73101411  1.52672073]
 [-0.64916132  1.62649961 -1.30902526 -1.31769898]
 [-0.40798684 -1.71807731  0.10766874  0.10451088]
 [-0.76974857  0.90980456 -1.36569302 -1.31769898]
 [ 0.79788559 -0.52358555  0.44767531  0.36309449]
 [ 1.03906007 -1.24028061  1.12768843  0.7509699 ]
 [ 1.15964732 -0.04578885  0.95768515  1.13884531]
 [-0.89033581  1.14870291 -1.36569302 -1.18840717]
 [ 0.19494938 -1.95697567  0.67434635  0.36309449]
 [ 0.5567111  -0.2846872   1.01435291  0.7509699 ]
 [ 0.91847283 -0.2846872   0.44767531  0.10451088]
 [ 2.2449325  -1.00138226  1.75103379  1.39742892]
 [-0.16681235  1.86539796 -1.19568974 -1.18840717]
 [-1.01092305  1.38760126 -1.36569302 -1.31769898]
 [-1.25209754 -0.04578885 -1.36569302 -1.18840717]
 [-0.76974857 -0.76248391  0.05

In [11]:
# Training with XW = Y

add1col = lambda x: np.concatenate((x, np.ones((x.shape[0], 1))), axis=1)

Ns, Nx = X_train.shape # (120, 4)

XX = add1col(X_train)
print(XX.shape)

YY = Y_train
print(YY.shape)

W = np.linalg.inv(XX.T.dot(XX)).dot(XX.T.dot(YY))
print(W)

def evaluate (X, W, Yd):
  '''
     X is np.array (Nsample, Nfeats);
     Yd is np.array (Nsample, Nonehot)
  '''
  x = add1col(X)
  yd = np.argmax(Yd, axis=1)
  y = np.argmax(x.dot(W), axis=1)
  print("CM:")
  print(confusion_matrix(yd, y))

evaluate(X_train, W, Y_train)
evaluate(X_test, W, Y_test)


(120, 5)
(120, 3)
[[ 0.06062508  0.05806364 -0.11868873]
 [ 0.10258458 -0.21112954  0.10854496]
 [-0.4070331   0.22897614  0.17805695]
 [-0.03856453 -0.30252108  0.34108561]
 [ 0.325       0.30833333  0.36666667]]
CM:
[[39  0  0]
 [ 0 22 15]
 [ 0  4 40]]
CM:
[[11  0  0]
 [ 0  6  7]
 [ 0  0  6]]


In [13]:
# With more non-linearity in X, for XW = Y

addSq1col = lambda x: np.concatenate((x, x**2, np.ones((x.shape[0], 1))), axis=1)
Ns, Nx = X_train.shape

XX = addSq1col(X_train)
print(XX.shape)

YY = Y_train
print(YY.shape)
W = np.linalg.inv(XX.T.dot(XX)).dot(XX.T.dot(YY))
print(W)


def evaluate(X, W, Yd):
  '''
     X is np.array (Nsamples, Nfeats),
     Yd is np.array (Nsamples, Nonehot)
  '''

  x = addSq1col(X)
  yd = np.argmax(Yd, axis=1)
  y = np.argmax(x.dot(W), axis = 1)
  print("CM:")
  print(confusion_matrix(yd, y))

evaluate(X_train, W, Y_train)
evaluate(X_test, W, Y_test)

(120, 9)
(120, 3)
[[ 0.02212851  0.1284788  -0.15060731]
 [ 0.02261966  0.00396191 -0.02658157]
 [-0.31265762 -0.00385162  0.31650925]
 [-0.05916917 -0.24649968  0.30566885]
 [-0.03340995  0.13233974 -0.09892979]
 [-0.00993575  0.01269958 -0.00276383]
 [ 0.21451737 -0.58556233  0.37104496]
 [ 0.04009892 -0.0947794   0.05468048]
 [ 0.1137294   0.84363575  0.04263485]]
CM:
[[39  0  0]
 [ 0 35  2]
 [ 0  2 42]]
CM:
[[11  0  0]
 [ 0 13  0]
 [ 0  0  6]]


In [18]:
# For under-determined system

add1col = lambda x: np.concatenate((x, np.ones((x.shape[0], 1))), axis=1)
ind = np.random.choice(range(X_train.shape[0]), size=12, replace=False)

XX= X_train[ind, :]
XX = add1col(XX)

YY = Y_train[ind, :]
YY = add1col(YY)

W = XX.T.dot(np.linalg.inv(XX.dot(XX.T)).T.dot(YY))     # X' = (XX') ^(-1)Y

print(W, XX.shape, YY.shape)

def evaluate(X, W, Yd):
  '''
     X is np.array (Nsamples, Nfeats),
     Yd is np.array (Nsamples, Nonehot)
  '''

  x = add1col(X)
  yd = np.argmax(Yd, axis=1)
  y = np.argmax(x.dot(W), axis=1)
  print("CM:")
  print(confusion_matrix(yd, y))

evaluate(X_train[ind, :], W, YY)
evaluate(X_test, W, Y_test)

                     

[[-1.14170893  1.2561653  -1.38162831 -0.44221603]
 [-0.18052177  2.46543779 -1.32396847  0.31971318]
 [ 0.26145432 -4.15378119  3.37342124 -0.09155385]
 [-1.32834135  4.20719101 -3.78105649  0.04373699]
 [ 0.8125     -2.125       2.25        0.84375   ]] (12, 5) (12, 4)
CM:
[[1 0 2 0]
 [0 0 6 1]
 [0 0 2 0]
 [0 0 0 0]]
CM:
[[ 5  4  2  0]
 [ 0  0 12  1]
 [ 0  3  2  1]
 [ 0  0  0  0]]


In [35]:
  # For under-determined system with sqaure input features

addSq1col = lambda x: np.concatenate((x, x**2, np.ones((x.shape[0], 1))), axis=1)
ind = np.random.choice(range(X_train.shape[0]), size=25, replace=False)
XX = X_train[ind,:]
XX = addSq1col(XX)
YY = Y_train[ind,:]
W = XX.T.dot(np.linalg.inv(XX.dot(XX.T)).T.dot(YY))

print(W, XX.shape, YY.shape)

def evaluate(X, W, Yd):
   '''
     X is np.array (Nsamples, Nfeats),
     Yd is np.array (Nsamples, Nonehot)
   '''
   x = addSq1col(X)
   yd = np.argmax(Yd, axis=1)
   y = np.argmax(x.dot(W), axis=1)
   print("CM:")
   print(confusion_matrix(yd, y))

evaluate(X_train[ind, :], W, YY)
evaluate(X_test, W, Y_test)

[[ 0.41615363 -1.32909283  1.04539054]
 [ 0.56232612  1.62905066 -0.72863816]
 [ 0.40222444 -0.67292253  0.37510919]
 [ 0.5113769   0.5353144  -0.28314039]
 [ 0.57311778 -5.89077362  2.42224719]
 [-0.40075492 -5.24185619  0.93005467]
 [ 1.24215614 -4.24169714  2.3487848 ]
 [ 0.85715498 -1.88040696  0.46110321]
 [ 0.77734375 -3.5390625   0.3125    ]] (25, 9) (25, 3)
CM:
[[ 0  0  6]
 [ 1  0  6]
 [ 2  0 10]]
CM:
[[ 0  0 11]
 [ 3  0 10]
 [ 0  0  6]]
